In [1]:
import xarray as xr
import rioxarray
import rasterio
from rasterio.enums import Resampling

# ==== User settings ====
input_nc = r"C:\Users\Ankit\Datasets_Forest_fire\compressed_cleaned_era5_2015_2016.nc"
output_tif = r"C:\Users\Ankit\Datasets_Forest_fire\ERA5_30m_stack_fixed.tif"
variables_to_process = ['t2m', 'd2m', 'u10', 'v10', 'tp']  # ERA5 variables
target_resolution = 30  # meters

# Open dataset
ds = xr.open_dataset(input_nc)
print("Dataset dimensions:", ds.dims)
print("Variables:", list(ds.data_vars))

target_crs = "EPSG:32644"  # UTM Zone (change if needed)

processed_bands = []

for var in variables_to_process:
    print(f"Processing variable: {var}")
    
    da = ds[var]
    
    # --- FIX: handle time properly ---
    if "time" in da.dims:
        da = da.mean(dim="time")   # ⬅ take mean over time
    elif "valid_time" in da.dims:
        da = da.mean(dim="valid_time")
    else:
        raise ValueError(f"Variable {var} has no time dimension! Check dataset.")
    
    # Attach CRS (ERA5 lat/lon WGS84)
    da = da.rio.write_crs("EPSG:4326")
    
    # Reproject to UTM & 30m resolution
    da_utm = da.rio.reproject(
        target_crs,
        resolution=target_resolution,
        resampling=Resampling.bilinear
    )
    
    processed_bands.append(da_utm)

# Stack into one multi-band GeoTIFF
stacked = xr.concat(processed_bands, dim="band")
stacked = stacked.assign_coords(band=range(1, len(variables_to_process) + 1))
stacked.rio.to_raster(output_tif)

print(f"✅ Multi-band raster saved at: {output_tif}")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Ankit\\OneDrive\\Desktop\\Datasets_Forest_fire\\compressed_cleaned_era5_2015_2016.nc'